# Connect to Stocktwits API

This notebook creates the stocktwits client to interact with the stocktwits API. It pulls the client_secret, the client_id, the redirect_url, and the access_token from the stocktwits_secrets.py file to authenticate. 

It uses the libraries Alpaca (for trading and making orders), stockquotes (for gettting the stock prices and metadata), and  

In [2]:
from stocktwits_secrets import client_secret, client_id, redirect_url, access_token
from alpaca import make_order, get_account_metadata, get_buying_power

import csv
import requests
import json
import os
import yfinance as yf
import time

import emojis
import datetime as dt
import stockquotes as sq

import pprint
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
from html5print import HTMLBeautifier

from yahoo_finance import Share
import yahoo_finance as yfin

In [3]:
# apple = sq.Stock('AAPL')
# apple.current_price

# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install stockquotes

119.99

In [10]:
# API Call to Stocktwits friends feed

#Use a requests proxy to get user messages from stock twits api
streams_friends_url = "https://api.stocktwits.com/api/2/streams/friends.json?access_token=" + access_token
streams_friends_url

# stocktwit_streams_url = "https://api.stocktwits.com/api/2/streams/symbol/" + SYMBOL + ".json?" + access_token
# stocktwit_streams_url

stocktwits_url = "https://api.stocktwits.com/api/2/oauth/authorize?client_id=" + client_id + "&response_type=code&redirect_uri=" + redirect_url + "&scope=read,watch_lists,publish_messages,publish_watch_lists,direct_messages,follow_users,follow_stocks"
response = requests.get(stocktwits_url)
response

<Response [200]>

## Configuration
Configuration consists of functions, and preset variables such a a sell threshold, order type, and quantity of shares. It also consists of urls for api get requests to both stocktwits and alpaca. 

In [ ]:
# 1. Authenticate
# 2. Run a buy alert check to see if the latest message was a buy alert DONE
# 3. If buy alert is true, connect to alpaca api and place order DONE
# 4. Write function that stores the price at the price the order was placed
# 5. Query stock price every second and compare it to original price. (While price < 1.05 * original_price: BUY)
# 6. If price is above a 2% increase. Sell.

In [71]:
# Variables
percent_threshold = 1.05 

# Order Parameters
buy_side = 'buy'
sell_side = 'sell'
market_type = 'market'
qty = '5'

In [30]:
 response_friends

{'response': {'status': 200},
 'cursor': {'more': True, 'since': 303476245, 'max': 303069864},
 'messages': [{'id': 303476245,
   'body': '@WhoDatSaints009',
   'created_at': '2021-03-13T03:54:48Z',
   'user': {'id': 1702156,
    'username': 'mrinvestorpro',
    'name': '88888888',
    'avatar_url': 'https://avatars.stocktwits.com/production/1702156/thumb-1576710991.png',
    'avatar_url_ssl': 'https://avatars.stocktwits.com/production/1702156/thumb-1576710991.png',
    'join_date': '2018-11-08',
    'official': False,
    'identity': 'User',
    'classification': [],
    'followers': 66150,
    'following': 43,
    'ideas': 66190,
    'watchlist_stocks_count': 289,
    'like_count': 95,
    'plus_tier': '',
    'premium_room': '',
    'trade_app': False},
   'source': {'id': 1149,
    'title': 'StockTwits for iOS',
    'url': 'http://www.stocktwits.com/mobile'},
   'conversation': {'parent_message_id': 303475882,
    'in_reply_to_message_id': 303475882,
    'parent': False,
    'repli

In [52]:
# Test Dictionary 
emojis.encode(':rotating_light:')

test_dict = {
    'messages': [{'body': "$AEO 🚨 buy alert 🚨"}]
    }

test_dict['messages'][0]['body']

'🚨'

In [205]:
#Functions

def buyAlertCheck(resp):
    #Use response from HTTP request and parse json response object
    
    count = 0
    body = resp['messages'][0]['body']
    list1 = body.split(" ")
    x = 'buy'
    emoj_light, emoj_fire = [emojis.encode(':rotating_light:'), emojis.encode(':fire:')]
    
    #If "buy alert" is found in text, then print the ticker symbol
    if x in body and (emoj_fire in body) or (emoj_light in body):
        #Print stock ticker from buy alert message
        dollar_sign = list1[0][:1]
        if dollar_sign == '$':
            print('Stock', list1[0][1:], 'was purchased on ', dt.time())
            ticker = list1[0][1:]
            
            print("Stock Ticker", list1[0])
            og_price = get_current_price(ticker)
            less_than = True
            curr_price = get_current_price(ticker)
            
            while less_than:
                
                if count == 0:
                    curr_price = og_price
                print("CURRENT PRICE:", curr_price)
                curr_price = get_current_price(ticker) if count == 0 else round(curr_price * 1.02,3)
                
                print(f'CURR PRICE: {curr_price}')
                #print(f"OG PRICE: {og_price} and CURR_PRICE: {curr_price} \n OG_MULT:")
                if count == 15:
                    less_than = False
                    
                elif (curr_price > (percent_threshold * og_price)) or (curr_price < og_price):
                    #Sell
                    print(f"Current Price: {curr_price}")
                    make_order(curr_price, qty, sell_side, market_type, 'day')
                    print(f"You have sold: {qty} shares of {ticker}")
                    less_than = False
                
                elif curr_price <= percent_threshold * og_price:
                    print(f"Current price is less than threshold: {percent_threshold}", less_than)
                
                    make_order(ticker, qty, buy_side, market_type, 'day') 
                    print(f"You have purchased: {qty} shares of {ticker} at {curr_price}")
                    count += 1
                    print("Count:", count)
                    less_than = True
                
                
            #Print buy alert message
            print("Buy Alert Message: ", "bought!" if list1[1:] == True else "not buying|")



        else:
            print('The first ticker was not a buy alert')
        #Pull stock ticker and invest in Alpaca API
    else:
        time_of_check = dt.date.today()#.strftime('%Y-%m-%d %r')
        print('The most recent message is NOT a buy alert on', dt.datetime.fromisoformat(resp['messages'][0]['created_at'][:-1]) , '\nToday\'s date:', time_of_check )
        print("Body of Message: ", body, time_of_check)
        return
    return 

def get_requests(url):
        #Return json object of messages
        """
            Parameters: url 
            Returns: response object from a get request
        """
        response = requests.get(url)
        response1 = json.loads(response.text)
        return response1

def get_current_price(tkr):
    """
        Parameters: tkr 
        Returns: price 
    """
    # 2. Store ticker in a variable
    var = tkr
    ticker_obj = sq.Stock(var)
    # 3. return price from ticker
    
    return ticker_obj.current_price

def get_position():
    
 

In [206]:
get_buying_power()

'158794.06'

In [207]:
get_current_price('AEO')

29.81

In [208]:
# 1. Authenticate
response_friends = get_requests(streams_friends_url)
#print(response_friends)

# body = response_friends['messages'][0]['body']
# list1 = body.split(" ")
# x = 'buy'

In [210]:
sq.Stock('AEO').current_price

29.81

In [212]:
buyAlertCheck(test_dict)

Stock AEO was purchased on  00:00:00
Stock Ticker $AEO
CURRENT PRICE: 29.81
CURR PRICE: 29.81
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 29.81
Count: 1
CURRENT PRICE: 29.81
CURR PRICE: 30.406
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 30.406
Count: 2
CURRENT PRICE: 30.406
CURR PRICE: 31.014
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 31.014
Count: 3
CURRENT PRICE: 31.014
CURR PRICE: 31.634
Current Price: 31.634
You have sold: 5 shares of AEO
Buy Alert Message:  not buying|


In [213]:
# 2. Run a buy alert check to see if the latest message was a buy alert

buy = True

while buy:
    buy = buyAlertCheck(response_friends)
    
    print("\n\nBuy Alert Status:", buy)
    time.sleep(3)
    print('waiting:', buy)
    
    if buy == None:
        buy = False
        
        
print('Script is done')

The most recent message is NOT a buy alert on 2021-03-13 03:54:48 
Today's date: 2021-03-13
Body of Message:  @WhoDatSaints009 2021-03-13


Buy Alert Status: None
waiting: None
Script is done


In [ ]:
# 3. If buy alert is true, connect to alpaca api and place order


In [21]:
#1. Application requests authorization
stocktwits_url = "https://api.stocktwits.com/api/2/oauth/authorize?client_id=" + client_id + "&response_type=code&redirect_uri=" + redirect_url + "&scope=read,watch_lists,publish_messages,publish_watch_lists,direct_messages,follow_users,follow_stocks"
response = requests.get(stocktwits_url)
response
#2. Authorization box with prompt user to approve request for authorization

#3. Stocktwits will redirect the user to the uri with a code as a query parameter: examples.com/?code=<code>

#4. Once the code is obtained, Use the code to request an access token using the client_id, client_secret, code, and redirect_uri
#https://api.stocktwits.com/api/2/oauth/token?client_id=<client id>&client_secret=<client secret>&code=<code>&grant_type=authorization_code&redirect_uri=http://www.example.com

#5. The response will return a username, user_id, access_token and scope. Use the access_token to 

<Response [200]>

In [9]:
#Request Token URL: Returns a user authorization token to make authenticated API calls
"https://api.stocktwits.com/api/2/oauth/token?client_id=" + client_id + "&client_secret=" + client_secret +"&code=" + "<code>" + "&grant_type=authorization_code&redirect_uri=" + redirect_url


'https://api.stocktwits.com/api/2/oauth/token?client_id=e5602262546c8dff&client_secret=8fa5bc8d87809ad35a3a562ef15a5ec35b1ece36&code=<code>&grant_type=authorization_code&redirect_uri=http://statsing.com/'

In [10]:
#CSV Variables
FILE_NAME = 'message_file_2_22.csv'
FIELDS = ['symbol', 'sentiment', 'message', 'message_id']

In [11]:
#Create a blank csv file with the name headers from the messages of stocktwits friends
def create_csv():
    #Header will include username, date_posted, followers, message
    file = open(FILE_NAME, 'a', newline='', encoding='utf-8')
    # DETERMINE WHERE TO START IF RESUMING SCRIPT
    if os.stat(FILE_NAME).st_size == 0:
        # OPEN FILE IN APPEND MODE AND WRITE HEADERS TO FILE
        last_message_id = [None, None, None, None]
        csvfile = csv.DictWriter(file, FIELDS)
        csvfile.writeheader()

In [12]:
SYMBOL = 'AAPL'

'https://api.stocktwits.com/api/2/streams/friends.json?access_token=5a84f31288f623b59883ed12c6c6a24ce67dab7a'

'https://api.stocktwits.com/api/2/streams/symbol/AAPL.json?5a84f31288f623b59883ed12c6c6a24ce67dab7a'

In [14]:
req_proxy = RequestProxy
req_proxy

http_request_randomizer.requests.proxy.requestProxy.RequestProxy

In [94]:
count = 0
api = 0
while True:
    #Get response 
    response = req_proxy.generate_proxied_request(stocktwit_streams_url)
    if response is not None:
        #Count the number of api hits
        api += 1
        
        #Record data from response
        response = json.loads(response.text)
        print('This is the', count, 'JSON object of the response: \n\n', response)
        if count == 50:
            break


NameError: name 'req_proxy' is not defined

In [19]:
# session_test = requests.Session()

In [90]:
# This a GET request for the streams/friends API
# session_test.request('GET', 
#                      stocktwit_streams_url, 
#                      data=body, #body is the message of the data. Will need to be converted to self.body
#                      timeout=(3.05, 27), 
#                      stream=True, 
#                      #auth=auth, 
#                      #verify=self.verify, 
#                      #proxies = self.proxies
#                     )

In [214]:
buyAlertCheck(test_dict)

Stock AEO was purchased on  00:00:00
Stock Ticker $AEO
CURRENT PRICE: 29.81
CURR PRICE: 29.81
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 29.81
Count: 1
CURRENT PRICE: 29.81
CURR PRICE: 30.406
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 30.406
Count: 2
CURRENT PRICE: 30.406
CURR PRICE: 31.014
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 31.014
Count: 3
CURRENT PRICE: 31.014
CURR PRICE: 31.634
Current Price: 31.634
You have sold: 5 shares of AEO
Buy Alert Message:  not buying|


In [215]:
buyAlertCheck(test_dict)

Stock AEO was purchased on  00:00:00
Stock Ticker $AEO
CURRENT PRICE: 29.81
CURR PRICE: 29.81
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 29.81
Count: 1
CURRENT PRICE: 29.81
CURR PRICE: 30.406
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 30.406
Count: 2
CURRENT PRICE: 30.406
CURR PRICE: 31.014
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 31.014
Count: 3
CURRENT PRICE: 31.014
CURR PRICE: 31.634
Current Price: 31.634
You have sold: 5 shares of AEO
Buy Alert Message:  not buying|


In [216]:
buyAlertCheck(test_dict)

Stock AEO was purchased on  00:00:00
Stock Ticker $AEO
CURRENT PRICE: 29.81
CURR PRICE: 29.81
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 29.81
Count: 1
CURRENT PRICE: 29.81
CURR PRICE: 30.406
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 30.406
Count: 2
CURRENT PRICE: 30.406
CURR PRICE: 31.014
Current price is less than threshold: 1.05 True
You have purchased: 5 shares of AEO at 31.014
Count: 3
CURRENT PRICE: 31.014
CURR PRICE: 31.634
Current Price: 31.634
You have sold: 5 shares of AEO
Buy Alert Message:  not buying|
